In [47]:
import os, sys
from numpy import mat
from pymatgen.ext.matproj import MPRester
import json

In [48]:
print(f"Python version info: {sys.version}")

Python version info: 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


In [49]:
API_TOKEN = os.environ.get("MPTOKEN")
# print(API_TOKEN)

## Helper Functions

In [50]:
def pprint(obj: object):
    print(json.dumps(obj, indent=2))
    
def save_findings(obj: object):
    with open("../TMR_Data.json", 'w') as fo:
        json.dump(obj, fo, indent=2)

## Actial data cleaning and aquisition

In [51]:
## Get possible ternery metal nitrats from the materials project.
# Search critea 
#   - All materials with nitrogen in its formula 
#   - these materials must have three elements
#   - then these materials must have a band gap value of less than 1 (logically a metal)

mpr = MPRester(API_TOKEN)

crt = {
    'elements': {'$all': ['N']},
    'nelements': 3,
    'band_gap': {'$lt': 1},
    # 'anonymous_formula': {"A": 1, "B": 1, "C": 3}
}
prt = ['material_id', 'pretty_formula', 'full_formula', 'band_gap', 'unit_cell_formula', 'elements']

materials = mpr.query(criteria=crt, properties=prt)

100%|██████████| 3153/3153 [00:11<00:00, 282.48it/s]


In [52]:
for i in materials[:5]: 
    print(i)

print("#"*50, f"\n\nFound {len(materials)} possible candidates")

{'material_id': 'mp-1013531', 'pretty_formula': 'Sr3BiN', 'full_formula': 'Sr3Bi1N1', 'band_gap': 0.0, 'unit_cell_formula': {'Sr': 3.0, 'Bi': 1.0, 'N': 1.0}, 'elements': ['Bi', 'N', 'Sr']}
{'material_id': 'mp-1013533', 'pretty_formula': 'Ba3PN', 'full_formula': 'Ba3P1N1', 'band_gap': 0.03489999999999993, 'unit_cell_formula': {'Ba': 3.0, 'P': 1.0, 'N': 1.0}, 'elements': ['Ba', 'P', 'N']}
{'material_id': 'mp-1013543', 'pretty_formula': 'Ca3AsN', 'full_formula': 'Ca3As1N1', 'band_gap': 0.0, 'unit_cell_formula': {'Ca': 3.0, 'As': 1.0, 'N': 1.0}, 'elements': ['As', 'Ca', 'N']}
{'material_id': 'mp-1013708', 'pretty_formula': 'Ba3SbN', 'full_formula': 'Ba3Sb1N1', 'band_gap': 0.0, 'unit_cell_formula': {'Ba': 3.0, 'Sb': 1.0, 'N': 1.0}, 'elements': ['Ba', 'Sb', 'N']}
{'material_id': 'mp-1018710', 'pretty_formula': 'Hf2TaN3', 'full_formula': 'Hf2Ta1N3', 'band_gap': 0.0, 'unit_cell_formula': {'Hf': 2.0, 'Ta': 1.0, 'N': 3.0}, 'elements': ['Hf', 'N', 'Ta']}
##########################################

Create a new materials object that includes the material type, ie metal or non metal

In [53]:
## Create a list that holds the different element groups
with open("../my_elements_data.json") as fo:
    my_elements_data = json.load(fo)


element_groups = list(set([i['GroupBlock'] for i in my_elements_data]))
print(element_groups)

['Noble gas', 'Actinide', 'Transition metal', 'Lanthanide', 'Post-transition metal', 'Alkaline earth metal', 'Nonmetal', 'Metalloid', 'Alkali metal', 'Halogen']


In [54]:
## Read my data for metals and non metals
with open("../my_elements_data.json") as fo:
    my_elements_data = json.load(fo)
    
# print(my_elements_data[:1])


my_classified_data_list = []
## Create new entry for every material object
for material in materials:
    my_classified_data = {}
    
    my_classified_data.update(material)
    
    # print(f"Processing - {material['pretty_formula']}")
    descriptive = []
    for element in material['elements']:
        # print(element)
        
        for i in my_elements_data:
            if i['Symbol'] == element:
                
                descriptive.append(f"{element} - {i['Name']} - {i['GroupBlock']}")
                # print(f"{element} - {i['Name']} is a {i['GroupBlock']}")
                # print(descriptive)
            
            my_classified_data['descriptive'] = descriptive
    
    my_classified_data_list.append(my_classified_data)
    # print(json.dumps(my_classified_data, indent=2))
    # print("#"*60, '\n\n')


pprint(my_classified_data_list[:2])

[
  {
    "material_id": "mp-1013531",
    "pretty_formula": "Sr3BiN",
    "full_formula": "Sr3Bi1N1",
    "band_gap": 0.0,
    "unit_cell_formula": {
      "Sr": 3.0,
      "Bi": 1.0,
      "N": 1.0
    },
    "elements": [
      "Bi",
      "N",
      "Sr"
    ],
    "descriptive": [
      "Bi - Bismuth - Post-transition metal",
      "N - Nitrogen - Nonmetal",
      "Sr - Strontium - Alkaline earth metal"
    ]
  },
  {
    "material_id": "mp-1013533",
    "pretty_formula": "Ba3PN",
    "full_formula": "Ba3P1N1",
    "band_gap": 0.03489999999999993,
    "unit_cell_formula": {
      "Ba": 3.0,
      "P": 1.0,
      "N": 1.0
    },
    "elements": [
      "Ba",
      "P",
      "N"
    ],
    "descriptive": [
      "Ba - Barium - Alkaline earth metal",
      "P - Phosphorus - Nonmetal",
      "N - Nitrogen - Nonmetal"
    ]
  }
]


Now We check if the material passes our search criteria of Metal-Metal-Nitrogen

In [55]:
# print(json.dumps(my_classified_data_list[:1], indent=2))

final_materials_list = []
## Metal Groups list
metal_groups = ['Transition metal', 'Actinide', 'Lanthanide', 'Alkali metal', 'Post-transition metal', 'Alkaline earth metal']

## Go through all the materials anch check their description for metal groups, metal group, and nitrogen
for material in my_classified_data_list:
    # pprint(material)
    
    # print(material['pretty_formula'], list(material['unit_cell_formula'].keys()), [i.split(' - ') for i in material['descriptive']])
    
    ## Represent the material formula in terma of its group
    element_group_formula = []
    for i in list(material['unit_cell_formula'].keys()):
        for j in [i.split(' - ') for i in material['descriptive']]:
            if j[0] == i:
                if len(element_group_formula) < 2:
                    element_group_formula.append(j[2])
                else:
                    element_group_formula.append(j[1])
    
    ## Check the material group-formula if matches M-M-N
    # print(material['pretty_formula'], element_group_formula)
    if (element_group_formula[0] in metal_groups) and (element_group_formula[1] in metal_groups) and (element_group_formula[2].lower() in "Nitrogen".lower()):
        # print(material['pretty_formula'], element_group_formula)
        final_materials_list.append(material)
    
    
    # break

# final_materials_list = set(final_materials_list)
pprint(final_materials_list[:2])
print(f"Found {len(final_materials_list)} possible candidates (unique ids = {len(set([mat['material_id'] for mat in final_materials_list]))})")
    
        
    

[
  {
    "material_id": "mp-1013531",
    "pretty_formula": "Sr3BiN",
    "full_formula": "Sr3Bi1N1",
    "band_gap": 0.0,
    "unit_cell_formula": {
      "Sr": 3.0,
      "Bi": 1.0,
      "N": 1.0
    },
    "elements": [
      "Bi",
      "N",
      "Sr"
    ],
    "descriptive": [
      "Bi - Bismuth - Post-transition metal",
      "N - Nitrogen - Nonmetal",
      "Sr - Strontium - Alkaline earth metal"
    ]
  },
  {
    "material_id": "mp-1018710",
    "pretty_formula": "Hf2TaN3",
    "full_formula": "Hf2Ta1N3",
    "band_gap": 0.0,
    "unit_cell_formula": {
      "Hf": 2.0,
      "Ta": 1.0,
      "N": 3.0
    },
    "elements": [
      "Hf",
      "N",
      "Ta"
    ],
    "descriptive": [
      "Hf - Hafnium - Transition metal",
      "N - Nitrogen - Nonmetal",
      "Ta - Tantalum - Transition metal"
    ]
  }
]
Found 1934 possible candidates (unique ids = 1934)


In [56]:
## Write findings to disk
final_materials_list_out = {}

my_outro = {}
my_outro['Total-Candidates'] = len(final_materials_list)
# my_outro['Search-Criteria'] = crt
# my_outro['Pulled-Properties'] = prt

final_materials_list_out['summary'] = my_outro
final_materials_list_out['candidates'] = final_materials_list

save_findings(final_materials_list_out)